# Implementação do NewHope com o *SageMath*

## Descrição do Exercício

Construir uma classe Python/SageMath que implemente o esquema KEM **NewHope** a concurso do NIST-PQC.

## Descrição da Implementação

Para a implementação houve a criação de um campo ciclotómico, através dum anel polinomial dum grupo finito de $1024$, logo ficando $R/[X^1024+1]$.

1. **Para a inicialização do algoritmo NewHope fez-se:**

    - Uma matriz identidade de sub-dimensão 4
    - Uma rede diagonal de inteiros a partir dessa matriz
    - Criar um meio vetor e modificar a última linha da matriz com ele
    - Criação do poliedro principal com as estruturas matemáticas anteriores

2. **Para a geração de noise/erro:**

    - Buscamos um *sample* polinomial da Distribuição Gaussiana

3. **Para a geração do sinal:**

    - Vamos fazer a divisão dos elementos pelo módulo tendo os coeficientes
    - Caso o vetor $V$ esteja no poliedro principal, utiliza-se. Senão vai-se à rede diagonal e usa-se o vetor mais próximo de $V$.
    - Sendo assim a distância, o sinal.

4. **Para a reconciliação é um processo análogo:**

    - Vamos fazer a divisão dos elementos pelo módulo tendo os coeficientes
    - Adicionamos $1$ caso a coordenada esteja no centro do poliedro, senão $0$
    - Temos assim uma chave.

## Resolução do Exercício

In [1]:
import itertools, numpy

from sage.stats.distributions.discrete_gaussian_polynomial import DiscreteGaussianDistributionPolynomialSampler
from sage.modules.free_module_integer import IntegerLattice
from sage.modules.diamond_cutting import calculate_voronoi_cell

dimension = 1024     # Grau dos Polinómios (Dimensão suportada pelo NewHope)
modulus = 12289      # Módulo (Valor de q)
sigma = 8/sqrt(2*pi) # Sigma (Com o Parâmetro de Distribuição de Ruído = 8)

# Anel Polinomial Quociente
R.<X> = PolynomialRing(GF(modulus))     # Anel polinomial
Y.<x> = R.quotient(X^(dimension) + 1)   # Campo Ciclotómico

subDimension = 4

# Função Auxiliar que converte do Intervalo [c0, c1, ..., c1023] para [(c0, c1, c2, c3), ..., (..., c1023)]
def grouped(iterable, n):
    return zip(*[iter(iterable)]*n)
    
class NewHope:
    
    # Inicialização Algortimo
    def initialize(self):

        # Criação da Matriz Identidade
        # Construir uma Rede diagonal de Inteiros a partir da Matriz Identidade
        identityMatrix = Matrix.identity(RR, subDimension)
        integerLattice = IntegerLattice(identityMatrix)

        # Construir um Half Vector (1/2)
        # Modificar última linha da Matriz Identidade
        halfVector = [1/2 for i in range(subDimension)]       
        identityMatrix[subDimension - 1] = halfVector
        
        # Criar Célula Voronoi a partir Matriz Modificada
        mainPolyhedron = calculate_voronoi_cell(identityMatrix).translation(halfVector) 
        
        # Retorna uma Lattice de Inteiros e um Poliedro de centro em (1/2, ..., 1/2)
        return (integerLattice, mainPolyhedron)
    
    def dbl(self, coefficient_vector):
        return  coefficient_vector + \
                vector( numpy.random.choice([0, 1], p=[0.5, 0.5]) * vector([1/(2*modulus) for _ in range(subDimension)]))

    # Geração do Erro
    def generateError(self):
        f = DiscreteGaussianDistributionPolynomialSampler(ZZ['x'], 5, sigma)()
        return Y(f)

    # Geração do Polinómio a partir de Elementos Random
    def generatePolynomial(self):
        return Y.random_element()

    # Geração do Signal
    def generateSignal(self, poly): 
        
        # Divide coeficientes pelo módulo
        coefficients = map(lambda x: RR(x) / modulus, poly.list())
        distances = []
        
        for v in grouped(coefficients, subDimension):
            
            v = self.dbl(vector(v))
            
            # Caso o ponto/vetor esteja no  Main Polyhedron, usa-se o centro do Main Polyhedron
            if mainPolyhedron.contains(vector(v)):
                distance = mainPolyhedron.center() - v
            # Caso contrário
            else:
                distance = integerLattice.closest_vector(v) - v
            distances.append(distance)

        return distances
    
    # Reconciliação
    def reconcile(self, poly, w):
        
        coefficients = map(lambda x: RR(x) / modulus, poly.list())
        key = []
        
        for difference, v in zip(w, grouped(coefficients, subDimension)):
            
            v = self.dbl(vector(v))
            coordinate = vector([round(point, 1) for point in (v + difference) ])
            key.append(1 if coordinate == mainPolyhedron.center() else 0)

        return "".join(map(str, key))

## Teste da Classe

Para teste da classe e algoritmo total fez-se:

- Inicialização do anel diagonal e o poliedro
- Cria-se uma matriz partilhada para os dois
- Instanciação dos valores do Bob (segredo, erro e o valor)
- Instanciação dos valores da Alice (segredo, erro e o valor)
- A chave da Alice sendo: $ valueBob * secretAlice + temp_error$
- A chave do Bob sendo: $ valueAlice * secretBob $
- Fazer a reconciliação destes polinómios e ver se são iguais.


**Teste e Verificação para o modo PKE-IND-CCA**

In [2]:
# Criação da Instância
newHope = NewHope()

# Lattice de Inteiros e Poliedro Principal
(integerLattice, mainPolyhedron) = newHope.initialize()

# Matriz Partilhada
shared = newHope.generatePolynomial()

# 1. Valores Bob
secretBob = newHope.generateError()
errorBob = newHope.generateError()
valueBob = shared * secretBob + errorBob

# 2. Valores Alice
secretAlice = newHope.generateError()
errorAlice = newHope.generateError()
valueAlice = shared * secretAlice + errorAlice

# Key Alice
temp_error = newHope.generateError()
keyAlice = valueBob * secretAlice + temp_error
w = newHope.generateSignal(keyAlice)
keyBinaryAlice = newHope.reconcile(keyAlice, w)

# Key Bob
keyBob = valueAlice * secretBob
keyBinaryBob = newHope.reconcile(keyBob, w)

if (keyBinaryBob == keyBinaryAlice):
    print "As chaves correspondem."
    print hex(int(keyBinaryBob, 2))
else:
    print "Erro na correspondência das chaves."

As chaves correspondem.
0x6407f9782c69ab7ee51a61a4235109af87ee0d0f40cc1d9e8899e1c0050e2d0L


## Observações Finais

- O algoritmo NewHope foi muito complicado de entender, dado que o próprio documento principal de envio não continha o algoritmo propriamente descrito, tal como acontecia para o NTRU-Prime;
- Recorreu-se à implementação deste algoritmo realizado através de outras linguagens de programação (incluindo o próprio Python), de forma a tentar compreender todo o processo algorítmico por detrás

## Referências

- GitHub, New Hope Key Exchange and Key Encapsulation https://github.com/Art3misOne/newhope (Acedido a 10 maio 2020)
- GitHub, PyNewHope https://github.com/scottwn/PyNewHope (Acedido a 10 maio 2020)
